In [1]:
#importing necessary libraries
import pandas as pd
import datetime
from bs4 import SoupStrainer
from bs4 import BeautifulSoup
import urllib.error, urllib.parse
import time
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import re
import random
import requests

In [2]:
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

class MyOpener(FancyURLopener, object):
    version = choice(user_agents)
myopener = MyOpener()


P:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
#get list of restaurant urls from csv file
url_list = pd.read_csv('Restaurant_Links.csv')
#print(url_list)

In [ ]:
counter = 0
error_urls = []
restaurant_name = []
restaurant_rating = []
restaurant_rating_count = []
restaurant_food_rating = []
restaurant_service_rating = []
restaurant_value_rating = []
large_review_restaurant_link = []
large_review_review_no = []
for url in url_list['Restaurant_names']:
    try:
        counter+=1
        print('Opening restautrant number:', counter)
        page=myopener.open(url)
        html = page.read().decode('utf-8')

        #BeautifulSoup takes a string object and parse out the document structure
        # and turn it into a BeautifulSoup object.
        soup = BeautifulSoup(html, "html5lib")

        # From the inspecting of the elements we can see that all the restaturant names, overall ratings, number of reviews are
        # embedded in <div id = "taplc_location_detail_header_restaurants_0">
        header = soup.find_all('div', attrs={"id": "taplc_location_detail_header_restaurants_0"})

        # Parse the restaurant name embedded in <h1 class = "heading_title">
        restaurant = header[0].find('h1', attrs={"class": "heading_title"})
        restaurant_name.append(restaurant.string)

        #Parse ratings
        rating = header[0].find('span', {"property": "ratingValue"})['content']
        restaurant_rating.append(rating)

        #Parse number of ratings
        count = header[0].find('span', {"property": "count"}).getText()
        restaurant_rating_count.append(count)


        # From the inspecting of the elements we can see that the food, value, service and atmosphere ratings are
        # embedded in <div id = "taplc_location_detail_header_restaurants_0">
        middle_section = soup.find_all('div', attrs={"class": "ui_columns is-multiline questionRatings"})

        keys = []
        classes = [value for element in middle_section[0].find_all(class_="ui_bubble_rating") for value in element["class"]]
        for mid_sec_inner in middle_section[0].find_all('span', attrs={"class": "text"}):
            keys.append(mid_sec_inner.getText())
        separate_ratings = []
        for value in classes:
            try:
                separate_ratings.append(float(value[value.find('bubble_')+7:])/10)
            except:
                pass
        restaurant_food_rating.append(separate_ratings[0])
        restaurant_service_rating.append(separate_ratings[1])
        restaurant_value_rating.append(separate_ratings[2])

        Reviewer_ID = []
        Review_title = []
        Review_date = []
        Reviews = []
        Overall_rating = []

        last_page=False

        while last_page == False:

            for review in soup.find_all("div", class_ = "review-container"):
                uid_str = review.find_all("div", class_='memberOverlayLink')
                if (len(uid_str) == 0):
                    Reviewer_ID.append(review.find_all("img", class_='basicImg')[0].get('data-mediaid'))
        #         print(uid_str)
                else:
                    Reviewer_ID.append(uid_str[0].get('id'))

                overall_rating = float(review.find_all("span", class_=re.compile("^ui_bubble_rating bubble_"))[0].attrs["class"][1][-2:])/10
        #         print(overall_rating)
                Overall_rating.append(overall_rating)

                rating_title = review.find_all("span", class_="noQuotes")[0].string
        #         print(rating_title)
                Review_title.append('"' + rating_title + '"')

                rating_date = review.find_all("span", {"class": "ratingDate"})[0]['title']
        #         print(rating_date)
                Review_date.append(rating_date)

                large_review = review.find_all('span', class_ = 'taLnk ulBlueLinks')
                if (len(large_review)):

                    #### Modify code here if you want to get full comments here itself

                    large_review_restaurant_link.append(url)
                    large_review_review_no.append(review.get('data-reviewid'))

                rating_content = review.find_all("p", class_="partial_entry")[0].contents[0]
                rating_content = re.sub('\n', ' ', rating_content)
        #         print(rating_content)
                Reviews.append('"' + rating_content + '"')


            try:
                # Open next page reviews
                pagination = soup.find_all("div", class_ = "prw_rup prw_common_north_star_pagination ")
                link = "https://www.tripadvisor.com"+pagination[0].find_all('a', {"class": "nav next taLnk ui_button primary"})[0]['href']
                page=myopener.open(link)
                html = page.read().decode('utf-8') #loading each reviews page
                soup = BeautifulSoup(html, "html5lib")

            except:
                last_page=True

        # Writing it to csv file
        rest_reviews = pd.DataFrame({
            'Overall_rating': Overall_rating,
            'Reviewer_ID': Reviewer_ID,
            'Review_title': Review_title,
            'Review_date': Review_date,
            'Reviews':Reviews
        })
        rest_reviews.to_csv('reviews_folder/' + str(counter)+'. ' +restaurant.string + '_reviews.csv', sep='|', encoding='utf-8', index=False)
        
    except:
        print('Error')
        error_urls.append(url)
error_urls.to_csv('error_urls.csv')

Opening restautrant number: 1
Opening restautrant number: 2
Opening restautrant number: 3
Opening restautrant number: 4
Opening restautrant number: 5
Opening restautrant number: 6
Opening restautrant number: 7
Opening restautrant number: 8
Opening restautrant number: 9
Opening restautrant number: 10
Opening restautrant number: 11
Opening restautrant number: 12
Opening restautrant number: 13
Opening restautrant number: 14
Opening restautrant number: 15
Opening restautrant number: 16
Opening restautrant number: 17
Opening restautrant number: 18
Opening restautrant number: 19
Opening restautrant number: 20
Opening restautrant number: 21
Opening restautrant number: 22
Opening restautrant number: 23
Opening restautrant number: 24
Opening restautrant number: 25
Opening restautrant number: 26
Opening restautrant number: 27
Opening restautrant number: 28
Opening restautrant number: 29
Error
Opening restautrant number: 30
Opening restautrant number: 31
Opening restautrant number: 32
Opening res

In [ ]:
#print(url)
#Reviews

In [ ]:
#Reviewer_ID
'reviews_folder/' + str(counter) +restaurant.string + '_reviews.csv'